In [2]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceBgeEmbeddings
from qdrant_client import QdrantClient
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain_huggingface import HuggingFaceEmbeddings
import re

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api = os.getenv("GROQ_API_KEY")

In [ ]:
# load the embedding model
# 1. BAAI/bge-small-en
# 2. BAAI/bge-base-en


# # LOCAL
# model_name = 'BAAI/bge-small-en'
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': False}

# embeddings= HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )
# print("Embedding Model Loaded........")

# Hugging Face embeddings

# Load BAAI/bge-base-en model
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en")

# Example usage: Generate embeddings for a text
text = "Agriculture analytics helps optimize crop yield."
embedding_vector = embeddings.embed_query(text)

# Print the embedding
print(embedding_vector[:5])  # Display first 5 values for brevity

print("Hugging Face Embedding Model Loaded........")

e:\ViitorCloud\Projects\RAG based Agentic AI for Motor Vehicles Act Compliances\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[-0.002015401842072606, 0.013313878327608109, 0.01592436619102955, -0.01759568229317665, 0.05512344837188721]
Hugging Face Embedding Model Loaded........


In [ ]:
url = "http://localhost:6333"
collection_name = "MVA_db_bge_base_en"

client = QdrantClient(
    url = url,
    prefer_grpc = False
)
print(client)
print("=============================")

Run this in powershell when there is connection error
docker compose -f .\docker-compose.yaml up --build -d

In [5]:
db = Qdrant(
    client = client,
    embeddings = embeddings,
    collection_name = collection_name
)

print(db)
print("=============================")

C:\Users\velan\AppData\Local\Temp\ipykernel_25452\2470458342.py:1: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  db = Qdrant(


In [6]:
query = """On March 5, 2025, at around 9:15 AM, a head-on collision occurred on the Eastern Express Highway in Mumbai when a two-wheeler, traveling on the wrong side , crashed into a sedan at high speed. The rider, identified as 24-year-old Arjun Verma, was trying to take a shortcut to avoid traffic when he suddenly entered the fast lane, colliding with an oncoming car driven by Priya Shah. The impact threw Arjun off his bike, causing severe head injuries , while Priya sustained minor bruises. Passersby called for help, and emergency services transported Arjun to the hospital. Police investigations confirmed that Arjun was at fault for violating traffic laws. Priya’s car repair costs were covered by Arjun’s third-party insurance, but he faced suspension of his driving license for repeated violations. The incident led to increased traffic surveillance and stricter enforcement against wrong-side driving in Mumbai."""

# db_result = db.similarity_search(
#     query=query,
#     k=3
# )

# for i in db_result:
#     print(i.page_content)
    # doc, score = i
    # print(f"\n====>> Score: {score}, Content: {doc.page_content}, Metadata: {doc.metadata}")

In [7]:
# Define llm model
llm = ChatGroq(model="llama3-8b-8192")

In [24]:
# Retrieval chain, Document chain
# Prompt template
prompt = ChatPromptTemplate.from_template(
    """
    you are a traffic police inspector, advicing and explaining the MVA sections.
So, based on the query:{input}, precisely filter the sections from the context
and describe section information, 
if the sections are interconnected to one another then include them too,
if any section is ommited then mention and highlight that,
provide punishment and fine only from "- Punishment and Fines" from context,
also apecify and explain if there is any state amendment if provided for particular section, 
pecisely based only on the provided context:{context}

output should be:

1. Section number:
2. Section Title:
3. Definition:
4. Detailed information:
5. State Amendments: If provided
6. Punishments and Fines: from "- Punishment and Fines:-" only if provided
"""
)

In [25]:
# Create document chain
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    you are a traffic police inspector, advicing and explaining the MVA sections.\nSo, based on the query:{input}, precisely filter the sections from the context\nand describe section information, \nif the sections are interconnected to one another then include them too,\nif any section is ommited then mention and highlight that,\nprovide punishment and fine only from "- Punishment and Fines" from context,\nalso apecify and explain if there is any state amendment if provided for particular section, \npecisely based only on the provided context:{context}\n\noutput shoul

In [26]:
# Input--->Retriever--->vectorstoredb

# Create Retrieve
retriever = db.as_retriever(similarity_score_threshold=0.3)
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [27]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Qdrant', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x0000027D2711C590>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    you are a traffic police inspector, advicing and explaining the MVA sections.\nSo, based on the query:{input}, pre

In [28]:
# Get the response from the LLM
response = retrieval_chain.invoke({"input":query})
response

{'input': 'On March 5, 2025, at around 9:15 AM, a head-on collision occurred on the Eastern Express Highway in Mumbai when a two-wheeler, traveling on the wrong side , crashed into a sedan at high speed. The rider, identified as 24-year-old Arjun Verma, was trying to take a shortcut to avoid traffic when he suddenly entered the fast lane, colliding with an oncoming car driven by Priya Shah. The impact threw Arjun off his bike, causing severe head injuries , while Priya sustained minor bruises. Passersby called for help, and emergency services transported Arjun to the hospital. Police investigations confirmed that Arjun was at fault for violating traffic laws. Priya’s car repair costs were covered by Arjun’s third-party insurance, but he faced suspension of his driving license for repeated violations. The incident led to increased traffic surveillance and stricter enforcement against wrong-side driving in Mumbai.',
 'context': [Document(metadata={'source': 'MVA_Data.md', '_id': 'f430d2c

In [29]:
for i in response['context']:
    print(i.page_content)
    print("\n========================\n")

MVA SECTION: 190. Using vehicle in unsafe condition.
    Definition: This section defines the offense of using a vehicle in an unsafe condition. It also covers driving a vehicle that violates road safety, noise, or air pollution standards. 
    (1) Any person who drives or causes or allows to be driven in any public place a motor vehicle or trailer while the vehicle or trailer has any defect, which such person knows of or could have discovered by the exercise of ordinary care and which is calculated to render the driving of the vehicle a source of danger to persons and vehicles using such place, shall be punishable with fine of one thousand five hundred rupees or, if as a result of such defect an accident is caused causing bodily injury or damage to property, with imprisonment for a term which may extend to three months, or with fine 12 of five thousand rupees, or with both and for a subsequent offence shall be punishable with imprisonment for a term which may extend to six months, or 

In [30]:
response['answer'].strip().split('\n\n')

['Based on the provided context, the following sections are applicable:',
 '**MVA SECTION: 190**',
 '1. Section number: 190\n2. Section Title: Using vehicle in unsafe condition\n3. Definition: This section defines the offense of using a vehicle in an unsafe condition. It also covers driving a vehicle that violates road safety, noise, or air pollution standards.\n4. Detailed information: The section states that any person who drives or causes or allows to be driven in any public place a motor vehicle or trailer while the vehicle or trailer has any defect, which such person knows of or could have discovered by the exercise of ordinary care and which is calculated to render the driving of the vehicle a source of danger to persons and vehicles using such place, shall be punishable with fine of one thousand five hundred rupees or, if as a result of such defect an accident is caused causing bodily injury or damage to property, with imprisonment for a term which may extend to three months, or

In [31]:
import re

# Extract text from the response dictionary
raw_text = response['answer']

# Remove Markdown-style formatting (e.g., '**' for bold)
clean_text = re.sub(r'\*\*', '', raw_text)

print(clean_text)

Based on the provided context, the following sections are applicable:

MVA SECTION: 190

1. Section number: 190
2. Section Title: Using vehicle in unsafe condition
3. Definition: This section defines the offense of using a vehicle in an unsafe condition. It also covers driving a vehicle that violates road safety, noise, or air pollution standards.
4. Detailed information: The section states that any person who drives or causes or allows to be driven in any public place a motor vehicle or trailer while the vehicle or trailer has any defect, which such person knows of or could have discovered by the exercise of ordinary care and which is calculated to render the driving of the vehicle a source of danger to persons and vehicles using such place, shall be punishable with fine of one thousand five hundred rupees or, if as a result of such defect an accident is caused causing bodily injury or damage to property, with imprisonment for a term which may extend to three months, or with fine 12 o